# How climate change perceptions vary around the world?

In [81]:
# Import libraries
import config
import pandas as pd
import numpy as np
import tweepy
from textblob import TextBlob
from textblob import Word
import re

# Change Pandas settings to show full text of a tweet
pd.set_option('display.max_colwidth', -1)

In [15]:
# Interacting with twitter's API
auth = tweepy.OAuthHandler(config.consumer_key, config.consumer_secret) 
auth.set_access_token(config.access_token, config.access_token_secret)
api = tweepy.API (auth, wait_on_rate_limit = True) 

In [46]:
# Clean the tweets
def clean_tweet(tweet):
    """ Utility function to clean a tweet by removing 
    links and special characters using regex.

    Parameters
    ----------
    tweet : string
        A text string that you want to parse and remove special characters.

    Returns
    -------
    The same tweet string without special characters removed.
    """
    
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())   

In [72]:
# List of tweets related to climate change
search_words = ("#ClimateChange", "Environment", "climate", "climatechange", "environmet")

def get_tweets_df(search_words):
    """Extracts tweets, finds sentiment score and returns tweets as a DataFrame 

    Parameters
    ----------
    search_words : list 
        A list of phrases to extract
    
    Returns
    -------
    Pandas DataFrame of tweets
    """
    res = []
    # Extract tweets for a given list of phrases
    for word in search_words:
        for tweet in tweepy.Cursor (api.search, 
                                    q = word + " -filter:retweets", 
                                    lang = "en").items(2):
            res.append(tweet)
   
    id_list = [tweet.id for tweet  in res]
    
    # Create tweets DataFrame
    data_set = pd.DataFrame(id_list, columns = ["id"])
    data_set["text"] = [tweet.text for tweet in res]
    data_set["len"]  = np.array([len(tweet.text) for tweet in res])
    data_set["created_at"] = [tweet.created_at for tweet in res]
    data_set["user_location"] = [tweet.author.location for tweet in res]
    data_set["Hashtags"] = [tweet.entities.get('hashtags') for tweet in res]
    
    # Use clean function to remove special characters
    data_set["text"] = np.array([ clean_tweet(tweet) for tweet in data_set['text'] ])
    
    # Add Sentiment score and class
    text = data_set["text"]
    for i in range(0,len(text)):
        textB = TextBlob(text[i])
        sentiment = textB.sentiment.polarity

        data_set.at[i, 'sentiment'] = sentiment
        if sentiment <0.00:
            SentimentClass = 'Negative'
            data_set.at[i, 'sentiment_class'] = SentimentClass 
        elif sentiment >0.00:
            SentimentClass = 'Positive'
            data_set.at[i, 'sentiment_class']= SentimentClass 
        else:
            SentimentClass = 'Neutral'
            data_set.at[i, 'sentiment_class']= SentimentClass
    
    return data_set
    

In [86]:
# Create tweets DataFrame
tweets_df = get_tweets_df(search_words)
tweets_df.head()

,id,text,len,created_at,user_location,Hashtags,sentiment,sentiment_class
0,1174331747426869248,Beneficial mutations in wild wheat may help crops cope with climatechange and increasing global temperatures,136,2019-09-18 14:38:01,Bristol | London UK,"[{'text': 'climatechange', 'indices': [60, 74]}]",0.050000,Positive
1,1174331685149773825,Follow 6 week journey with as she reports firsthand on project preparation by 100s of,140,2019-09-18 14:37:46,"DC, NYC, ATL, STL, TEX & CA",[],0.000000,Neutral
2,1174331779106430976,Climate change is an individual responsibility However without systemic changes at government level the future loo,140,2019-09-18 14:38:08,london,[],0.000000,Neutral
3,1174331772412342272,So California leads the fight against climate change and also leads the fight for the right to destroy the environm,140,2019-09-18 14:38:07,New York City,[],0.042857,Positive
4,1174331798727340033,Walsall Council declares climate emergency but with a 30 year deadline LDReporter,117,2019-09-18 14:38:13,Birmingham,"[{'text': 'LDReporter', 'indices': [99, 110]}]",0.000000,Neutral
